In [188]:
import healpy as hp
import numpy as np
import os
from lsst.daf.persistence import Butler
repo_dir = '/datasets/hsc/repo/rerun/DM-13666/WIDE'
butler = Butler(repo_dir)
#/datasets/hsc/repo/rerun/DM-13666/DEEP/deepCoadd-results/HSC-R

In [189]:
nside = 512
npix = hp.nside2npix(nside)
print('resolution %e sq degrees' % (4.0*np.pi*(180.0/np.pi)**2/npix))

resolution 1.311396e-02 sq degrees


In [190]:
pixel_counts = np.zeros(npix, dtype=float)

In [191]:
sky_map = butler.get('deepCoadd_skyMap')

In [192]:
catalog_name = 'deepCoadd_forced_src'
ref_name = 'deepCoadd_ref'
filter_name = 'HSC-R'

In [193]:
tract_root = os.path.join(repo_dir, 'deepCoadd-results', filter_name)
tract_list = os.listdir(tract_root)
tract_list = np.array(tract_list).astype(int)

In [ ]:
valid_data_id = []
for tract_id in tract_list:
    tract = sky_map[tract_id]
    patches = tract.getNumPatches()
    for ip1 in range(patches[0]):
        for ip2 in range(patches[1]):
            dataId = {'filter':filter_name,
                      'tract':tract_id,
                      'patch':'%d,%d' % (ip1, ip2)}
            if not butler.datasetExists(catalog_name, dataId=dataId):
                continue
            else:
                valid_data_id.append(dataId)

In [ ]:
ref_cat = butler.get(ref_name, dataId=valid_data_id[0])
src_cat = butler.get(catalog_name, dataId=valid_data_id[0])

In [ ]:
len(ref_cat)

In [ ]:
len(src_cat)

In [ ]:
np.testing.assert_array_equal(ref_cat['coord_ra'], src_cat['coord_ra'])

In [ ]:
np.testing.assert_array_equal(ref_cat['coord_dec'], src_cat['coord_dec'])

In [ ]:
ref_cat['detect_isPrimary']

In [ ]:
ref_cat.schema.find('base_PsfFlux_flux')

In [ ]:
#src_cat.schema.getNames()

In [ ]:
src_val = src_cat['base_ClassificationExtendedness_value']
ref_val = ref_cat['base_ClassificationExtendedness_value']
flux_flag = ref_cat['modelfit_CModel_flag']
src_psf_flux = src_cat['base_PsfFlux_flux']
src_model_flux = src_cat['modelfit_CModel_flux']
ref_psf_flux = ref_cat['base_PsfFlux_flux']
ref_model_flux =ref_cat['modelfit_CModel_flux']

src_ratio = src_psf_flux/src_model_flux
ref_ratio = ref_psf_flux/ref_model_flux
for ii in range(len(src_cat)):
    if src_val[ii] != ref_val[ii] and not np.isnan(src_val[ii]) and \
    not np.isnan(ref_val[ii]):
        print(src_val[ii], ref_val[ii],
              src_ratio[ii], ref_ratio[ii])

In [ ]:
help(hp.ang2pix)

In [ ]:
import time
ct = 0
pixel_counts = np.zeros(npix, dtype=float)
t_start = time.time()
for dataId in valid_data_id:
    ref_cat = butler.get(ref_name, dataId=dataId)
    is_primary = ref_cat['detect_isPrimary']
    is_star = ref_cat['base_ClassificationExtendedness_value'] < 0.99
    valid = is_primary & is_star
    data = ref_cat[valid]
    pixel_number = hp.ang2pix(nside,
                              np.degrees(data['coord_ra']),
                              np.degrees(data['coord_dec']),
                              lonlat=True)

    l_pixel_dexes, l_pixel_counts = np.unique(pixel_number,
                                              return_counts=True)
    pixel_counts[l_pixel_dexes]+=l_pixel_counts
    ct += 1
    if ct %10 ==0:
        duration = time.time()-t_start
        predict = len(valid_data_id)*duration/ct
        print('ran %d of %d dataIds; took %e sec; total should take %e sec' %
              (ct, len(valid_data_id), duration, predict))

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
valid_pixels = np.where(pixel_counts>0.0)

In [ ]:
ra_arr, dec_arr = hp.pix2ang(nside, range(len(pixel_counts)),lonlat=True)

In [ ]:
valid_ra = ra_arr[valid_pixels]
valid_dec = dec_arr[valid_pixels]
valid_counts = pixel_counts[valid_pixels]

ra_min = valid_ra.min()
ra_max = valid_ra.max()
dec_min = valid_dec.min()
dec_max = valid_dec.max()

In [ ]:
print(ra_min,ra_max)
print(dec_min,dec_max)

In [ ]:
plt.figsize=(30,30)

plt.scatter(valid_ra, valid_dec, c=valid_counts)
plt.xlabel('ra')
plt.ylabel('dec')
plt.colorbar()

In [ ]:
cart_view = hp.mollview(pixel_counts, cbar=False,
                        #lonra=[ra_min,ra_max],
                        #latra=[dec_min,dec_max],
                        flip='astro',
                        return_projected_map=True)


plt.imshow(cart_view, extent=[ra_min, ra_max, dec_min, dec_max],
           origin='lower', interpolation='none')
plt.colorbar()
ax = plt.gca()
plt.xlabel('ra')
ax.set_ylabel('dec')
xticks = np.arange(ra_min, ra_max, 0.1*(ra_max-ra_min))
print(cart_view.shape)
print(cart_view.shape[0]*cart_view.shape[1], npix)
print(cart_view)

In [ ]:
hp.mollview(pixel_counts, cbar=False,
                        flip='astro',
                        return_projected_map=True)
hp.graticule()
ax = plt.gca()
im = ax.get_images()[0]
plt.colorbar(im)

In [ ]:
len(valid[0])

In [ ]:
len(pixel_counts)

In [ ]:
np.median(pixel_counts[valid])

In [ ]:
#help(hp.gnomview)

In [ ]:
help(hp.projaxes)

In [ ]:
hp.nside2npix(16)

In [ ]:
help(hp.graticule)